# Digital Filter FIR Type I
ここでは，Type Iのみを扱う<br>
Type I
This function computes the coefficients of a finite impulse response filter. The filter will have linear phase; it will be Type I if numtaps is odd and Type II if numtaps is even.

Type II filters always have zero response at the Nyquist frequency, so a ValueError exception is raised if firwin is called with numtaps even and having a passband whose right end is at the Nyquist frequency. <br>

see https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html


#### FIRフィルタの係数を求める
scipy.signal.firwin  https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html <br>
FIRフィルタの伝達関数は分子だけであるから，左辺は分子の係数を与えるbのみを置く<br>
b = scipy.signal.firwin(numtaps, fc_L, pass_zero=True)          # Low-pass<br>
b = scipy.signal.firwin(numtaps, fc_H, pass_zero=False)         # High-pass<br>
b = scipy.signal.firwin(numtaps, [fc_L, fc_H], pass_zero=False) # Band-pass<br>
b = scipy.signal.firwin(numtaps, [fc_L, fc_H])                  # Band-stop<br>

#### 離散時間系伝達関数の周波数応答　scipy.signal.freqz
https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqz.html <br>
離散時間系ゆえ，左辺のwには，w = 0 - pi [rad/sample] が代入される

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(123)
FLAG_fig = False

#### 注意：アナログフィルタの場合と異なり，下記の周波数の単位は[Hz]で与える

In [ ]:
fc_L   = 3.  # カットオフ周波数 [Hz]
fc_H   = 7.  # 上側カットオフ周波数 [Hz]
fsmp   = 50. # サンプリング周波数 [Hz]
fnyq   = fsmp/2.0 # ナイキスト周波数 [Hz]

Ntaps =127 # タップ数，Type I より奇数とする

#### ローパスフィルタ

In [ ]:
b1 = signal.firwin(numtaps=Ntaps, cutoff=fc_L,
                   window='hamming', pass_zero=True, fs = fsmp)

In [ ]:
w, h = signal.freqz(b1)
gain = 20*np.log10(abs(h))

#### 横軸 [rad/sample] and [Hz]
w[rad/sample] = pi と　f[Hz] = fnyq　は等価である。<br>
また，w = 2*pi*f より，スクリプトでf = (w/np.pi)*fnyq　とすれば[Hz]を得られる。

In [ ]:
fig, (ax1, ax2)  = plt.subplots(nrows=2, figsize=(6,8))

# Plot Gain[dB]  vs [rad/sample]
ax1.plot(w, gain, c='b')
ax1.axhline(-3, color='g', ls='--') # -3[dB]
#ax1.set_ylim(-50, 2)

ax1.grid(which = "both", axis='both', ls="--")
ax1.set_xlabel('Omega [rad / sample]')
ax1.set_ylabel('Gain [dB]')

# x-axis [Hz]
f = (w/np.pi)*fnyq

# Gain - Linear
ax2.plot(f, abs(h), c='b')
ax2.axhline(1/np.sqrt(2), color='g', ls='--') # 1/sqrt(2)

ax2.grid(which = "both", axis='both', ls="--")
ax2.set_xticks(np.arange(f.min(),f.max(),1))

ax2.set_xlabel('f [Hz]')
ax2.set_ylabel('Gain [Linear]')

# Phase
ax21 = ax2.twinx()
phase = np.unwrap(np.angle(h))  # radian

ax21.plot(f, phase, c='c')
ax21.set_ylabel('Phase [rad]')

if FLAG_fig: plt.savefig('fig_DFIL_FIR_Design_Lowpass.png')
plt.show()

#### バンドパスフィルタ

In [ ]:
b2 = signal.firwin(numtaps=Ntaps, cutoff=[fc_L, fc_H], 
                   window='hamming', pass_zero=False, fs = fsmp)
w, h = signal.freqz(b2)
gain = 20*np.log10(abs(h))

In [ ]:
fig, (ax1, ax2)  = plt.subplots(nrows=2, figsize=(6,8))

# Plot Gain[dB] vs. [rad/sample]
ax1.plot(w, gain, c='b')
ax1.axhline(-3, color='g', ls='--') # -3[dB]

ax1.grid(which = "both", axis='both', ls="--")
ax1.set_xlabel('Omega [rad / sample]')
ax1.set_ylabel('Gain [dB]')

# x-axis [Hz]
f = (w/np.pi)*fnyq

# Gain - Linear
ax2.plot(f, abs(h), c='b')
ax2.axhline(1/np.sqrt(2), color='g', ls='--') # 1/sqrt(2)

ax2.grid(which = "both", axis='both', ls="--")
ax2.set_xticks(np.arange(f.min(),f.max(),1))
ax2.set_xlabel('f [Hz]')
ax2.set_ylabel('Gain [Linear]')

# Phase
ax21 = ax2.twinx()
phase = np.unwrap(np.angle(h))  # radian

ax21.plot(f, phase, c='c')
ax21.set_ylabel('Phase [rad]')

if FLAG_fig: plt.savefig('fig_DFIL_FIR_Design_Bandpass.png')
plt.show()

# フィルタの効用を見る
設計したローパスフィルタ，バンドバスフィルタの性能を見るため，二つの周波数成分を持つ信号に雑音が重畳した観測値を用いる。<br>
scipy.signal.lfilter   https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.lfilter.html

In [ ]:
#観測信号
frq1, frq2 = 1.0, 5.0
Num = 256    # データ数
dt  = 1/fsmp      # サンプリング時間
t = np.linspace(0, Num-1, Num)*dt
y1 = np.sin(2*np.pi*frq1*t)
y2 = np.sin(2*np.pi*frq2*t)
y = y1 + y2 + 0.2*np.random.randn(t.size) # 雑音が重畳

In [ ]:
#ローパスフィルタによるフィルタリング
y_filt1 = signal.lfilter(b1,1,y)

In [ ]:
fig = plt.subplots(figsize=(8,4))

plt.plot(t,y, 'k', label='observed')
plt.plot(t,y_filt1, 'g', label='filtered')
#下記の (t-1.6)は試行錯誤的に見出した
plt.plot(t, np.sin(2*np.pi*frq1*t-1.6), c='r', ls='--', label='y1 with lag')

plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Low pass')
plt.legend()

if FLAG_fig: plt.savefig('fig_DFIL_FIR_Design_Filtered1.png')
plt.show()

In [ ]:
#バンドバスフィルタによるフィルタリング
y_filt2 = signal.lfilter(b2,1,y)

fig = plt.subplots(figsize=(8,4))

plt.plot(t,y_filt2, 'g', label='filtered')
#下記の (t-1.5)は試行錯誤的に見出した
plt.plot(t, np.sin(2*np.pi*frq2*t-1.5), c='r', ls='--', label='y2 with lag')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Band pass')
plt.legend()

if FLAG_fig: plt.savefig('fig_DFIL_FIR_Design_Filtered2.png')
plt.show()